# Part I: Different Model Specs


### A. Regression without regularization


### B. Ridge regression


### C. Lasso Regression


### D. Elastic Net


# Part II. Variable Selection


# Part III. Discussion


### A. Ridge


### B. LASSO


### C. Elastic Net


# Part IV: Final Model
